## Module 4, Activity 4: Radar Plots

In this activity, we'll continue using the penguins dataset to make radar plots. Again, we'll be using the fantastic Plotly library for our visualisations. Let's get started.

In [112]:
# import packages/libraries
import plotly
import plotly.graph_objects as go
import pandas as pd

# load dataset
df = pd.read_csv("data/penguins.csv")
df = df[["species","bill_length_mm", "bill_depth_mm", "flipper_length_mm", "body_mass_g"]]

Let's make a radar plot comparing the physiological characteristics of each of the three penguin species. For each species, we have over 100 observations. But a radar plot can only take a single value for each characteristic. So, we'll first take the average of each physiological characteristic, grouped by species.

In [113]:
ave_df = df.groupby("species").mean()
ave_df

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
species,,,,
Adelie,38.791391,18.346358,189.953642,3700.662252
Chinstrap,48.833824,18.420588,195.823529,3733.088235
Gentoo,47.504878,14.982114,217.186992,5076.016260


We'll use the **[Scatterpolar()](https://plotly.com/python/reference/scatterpolar/)** function from Plotly to create our radar plots. The **Scatterpolar()** function can take a Pandas data frame and uses the column names to define a radar chart axis and axis values. and column name of dataframe which can be used for radar chart axis and axis values. Let's make our first radar plot, with just data for the Adelie species.

In [114]:
fig = go.Figure(data=go.Scatterpolar(
                    r=ave_df.loc["Adelie"].values, # Length of each arm
                    theta=ave_df.columns, # Radar plot arms
                    fill='toself', # Fill shape defined by the endpoints of each arm
                    fillcolor = "red", # Fill colour
                    opacity=0.6,  # Opacity of fill
                    line=dict(color="red"))) # Line colour around shape


fig.update_layout(
    title="Palmer Archipelago Penguin Physiology", # Figure title
)

fig.show()

We have a problem with the scale. The numbers for body mass are in the 1,000s, while bill depth is in the 10s. We need to rescale our data to address this. Remember, we want to compare the physiology of the different penguins. 
At present, radar plots where each axis is independent can't be made with Plotly, but hopefully this issue will be [resolved soon](https://github.com/plotly/plotly.js/issues/3086). You can do this with Matplotlib, but the code is [very cumbersome](https://stackoverflow.com/questions/24659005/radar-chart-with-multiple-scales-on-multiple-axes). In the meantime, let's just divide each column of ave_df by its maximum value. This will normalise the column values to all be between 0 and 1.

In [115]:
ave_df_norm = ave_df/ave_df.max()
ave_df_norm

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
species,,,,
Adelie,0.794355,0.995970,0.874609,0.729049
Chinstrap,1.000000,1.000000,0.901636,0.735437
Gentoo,0.972786,0.813335,1.000000,1.000000


Let's try our Adelie radar plot again, but with the normalised data.

In [116]:
fig = go.Figure(data=go.Scatterpolar(
                    r=ave_df_norm.loc["Adelie"].values, # Length of each arm
                    theta=ave_df_norm.columns, # Radar plot arms
                    fill='toself', # Fill shape defined by the endpoints of each arm
                    fillcolor = "red", # Fill colour
                    opacity=0.6,  # Opacity of fill
                    line=dict(color="red"))) # Set line colour

fig.update_layout(
    title="Palmer Archipelago Penguin Physiology", # Figure title
)

fig.show()

That's better. Let's clean up our labels before continuing. Since we've rescaled our variables, we should relabel our radial axis ticks as well, since 0, 0.2, 0.4, 0.6, 0.8, 1 don't mean much. Let's also switch off the labels that pop up when you hover over each point on the radar plot, since they also don't mean much because of the normalisation.

In [134]:
fig = go.Figure(data=go.Scatterpolar(
                    r=ave_df_norm.loc["Adelie"].values, # Length of each arm
                    theta=ave_df_norm.columns, # Radar plot arms
                    fill='toself', # Fill shape defined by the endpoints of each arm
                    fillcolor = "red", # Fill colour
                    opacity=0.6,  # Opacity of fill
                    line=dict(color="red"))) # Set line colour

fig.update_layout(
    title="Palmer Archipelago Penguin Physiology", # Figure title
    hovermode=False, # Turn off hover labels (since they aren't very useful because of normalisation)
    polar = dict( # Create a dictionary of variables for layout.polar to read and update our figure with
                radialaxis = dict(tickvals = [0,1], # At radial axis ticks 0,1...
                        ticktext = ["", "Max"]),           # Label 0 at 0, and "Max" at 1
                 
    angularaxis = dict(rotation = 45, # Rotate our axis labels around radar plot by 45 degrees (to move away from radial axis)
            # Clean up the names of our variables
            tickvals=["bill_depth_mm", "bill_length_mm", "body_mass_g", "flipper_length_mm"],
            ticktext=["Bill Depth, mm", "Bill Length", "Body Mass, g", "Flipper Length, mm"]
            ) # End angularaxis dictionary
            ) # End polar dictionary
            ) # End update_layout function

fig.show()

**Exercise:** Above, we're calling variables in the [**layout.polar**](https://plotly.com/python/reference/layout/polar/) function. Experiment with these variables, changing axis tick labels, and the rotation of our axis labels.

We want to compare the three penguin species, so let's not overlay the radar plots for all three species on a single axes. We can do this with a **for loop**.

In [136]:
fig = go.Figure()

species = df.species.unique() # Unique species labels
colors=["red", "green", "blue"] # Colours for each species

for i in range(len(species)): # Loop over species, adding a radar plot for each to our figure
    fig.add_trace(go.Scatterpolar(
                    r=ave_df_norm.loc[species[i]].values, # Length of each arm
                    theta=ave_df_norm.columns, # Radar plot arms
                    fill='toself', # Fill shape defined by the endpoints of each arm
                    fillcolor = colors[i], # Fill colour
                    name = species[i], # Name of current radar plot
                    opacity=0.3,  # Opacity of fill
                    line=dict(color=colors[i]))) # Set line colour

fig.update_layout(
    title="Palmer Archipelago Penguin Physiology", # Figure title
    hovermode=False, # Turn off hover labels (since they aren't very useful because of normalisation)
    polar = dict( # Create a dictionary of variables for layout.polar to read and update our figure with
                radialaxis = dict(tickvals = [0,1], # At radial axis ticks 0,1...
                        ticktext = ["", "Max"]),           # Label 0 at 0, and "Max" at 1
                 
    angularaxis = dict(rotation = 45, # Rotate our axis labels around radar plot by 45 degrees (to move away from radial axis)
            # Clean up the names of our variables
            tickvals=["bill_depth_mm", "bill_length_mm", "body_mass_g", "flipper_length_mm"],
            ticktext=["Bill Depth, mm", "Bill Length", "Body Mass, g", "Flipper Length, mm"]
            ) # End angularaxis dictionary
            ) # End polar dictionary
            ) # End update_layout function


fig.show()

**Exercise:** Create a radar plot using the Penguins dataset, but grouping individuals by sex, not species.